In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357288 sha256=511fac54b16c539177034725b12f79cb9dc4029a85ebb217b72fdd551180459b
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import os

EDA on Dataset


In [3]:
import gdown
import pandas as pd

# File ID from Google Drive
file_url = 'https://drive.google.com/uc?id=1-vElZA5bxPzKiq41wwpM7H8LbSFuIWiC'
output_file = 'combined_data_1.txt'

# Download the file
gdown.download(file_url, output_file, quiet=False)

# Load the data
data = pd.read_csv(
    output_file,
    header=None,
    names=['Cust_Id', 'Rating'],
    usecols=[0, 1]
)

# Convert it to a DataFrame
df = pd.DataFrame(data)

# Display the first few rows to confirm the structure
print(df.head())

Downloading...
From (original): https://drive.google.com/uc?id=1-vElZA5bxPzKiq41wwpM7H8LbSFuIWiC
From (redirected): https://drive.google.com/uc?id=1-vElZA5bxPzKiq41wwpM7H8LbSFuIWiC&confirm=t&uuid=ec0a6b13-77ec-4f0e-9044-79ad91950a1b
To: /content/combined_data_1.txt
100%|██████████| 495M/495M [00:05<00:00, 82.7MB/s]


   Cust_Id  Rating
0       1:     NaN
1  1488844     3.0
2   822109     5.0
3   885013     4.0
4    30878     4.0


In [4]:
import pandas as pd
import gdown

# Define the file URL and output filename
file_url = 'https://drive.google.com/uc?id=1LNdh92ktYJC4JlYFDD0E5rkNDdl4_hGp'
output_file = 'movie_titles.csv'

# Download the file
gdown.download(file_url, output_file, quiet=False)

# Load the Movie Titles dataset
df_title = pd.read_csv(
    output_file,
    encoding='ISO-8859-1',
    header=None,
    usecols=[0, 1, 2],
    names=['Movie_Id', 'Year', 'Name']
)
df_title.set_index('Movie_Id', inplace=True)

# Display the first few rows of the movie titles dataset
print("Movie Titles Dataset:")
print(df_title.head(5))

Downloading...
From: https://drive.google.com/uc?id=1LNdh92ktYJC4JlYFDD0E5rkNDdl4_hGp
To: /content/movie_titles.csv
100%|██████████| 578k/578k [00:00<00:00, 8.78MB/s]

Movie Titles Dataset:
            Year                          Name
Movie_Id                                      
1         2003.0               Dinosaur Planet
2         2004.0    Isle of Man TT 2004 Review
3         1997.0                     Character
4         1994.0  Paula Abdul's Get Up & Dance
5         2004.0      The Rise and Fall of ECW


Cleaning the Data

In [5]:
# Creating Dataframe with all movie id rows
df_nan=pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating']==True].reset_index()

In [6]:
df_nan

,index,Rating
0,0,True
1,548,True
2,694,True
3,2707,True
4,2850,True
...,...,...
4494,24046714,True
4495,24047329,True
4496,24056849,True
4497,24057564,True


In [7]:
# Assigning the moveid to relevant index
df['Movie_Id'] = 0
for i in range(1,len(df_nan)):
    prev_index = df_nan['index'].iloc[i-1]
    curr_index = df_nan['index'].iloc[i]
    df['Movie_Id'].iloc[prev_index:curr_index] = i
   # print(i   ,prev_index,curr_index)

In [8]:
# Removing the NaN rows as movie id is seperated
df = df[pd.notnull(df['Rating'])]
# Changing the column to int datatype
df['Movie_Id'] = df['Movie_Id'].astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24053764 entries, 1 to 24058262
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_Id   int64  
 1   Rating    float64
 2   Movie_Id  int64  
dtypes: float64(1), int64(2)
memory usage: 734.1 MB


In [10]:
df.head()

,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [11]:
# Removing the inactive records by elminating records < 80 percentile
f = ['count','mean']
for col in ['Cust_Id','Movie_Id']:
    df_cnt = df.groupby(col)['Rating'].agg(f)
    df_cnt.index = df_cnt.index.map(int)
    threshold_val = round(df_cnt['count'].quantile(.8))
    if col == 'Cust_Id':
        df_cust = df_cnt
        cust_drop_id = df_cust [ df_cust['count'] < threshold_val ].index
    else:
        df_movie = df_cnt
        movie_drop_id = df_movie [ df_movie['count'] < threshold_val ].index
    print(f'The threshold Value for {col} :',threshold_val)

The threshold Value for Cust_Id : 79
The threshold Value for Movie_Id : 3884


In [12]:
print('Original Shape: {}'.format(df.shape))
df = df[~df['Cust_Id'].isin(cust_drop_id)]
df = df[~df['Movie_Id'].isin(movie_drop_id)]
print('After Trim Shape: {}'.format(df.shape))

Original Shape: (24053764, 3)
After Trim Shape: (13528427, 3)


In [13]:
df_title

,Year,Name
Movie_Id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW
...,...,...
17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17767,2004.0,Fidel Castro: American Experience
17768,2000.0,Epoch


In [14]:
!pip install scikit-surprise
from surprise import Reader, Dataset

K-means Clustering

In [15]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets (ensure these are already loaded as per your previous code)
# Movie titles and ratings data should already be loaded in `df_title` and `df`

# Step 1: Create a User-Movie Matrix
df_sparse_mat = pd.pivot_table(df, values='Rating', index='Cust_Id', columns='Movie_Id').fillna(0)
print("Sparse Matrix Shape:", df_sparse_mat.shape)

# Step 2: Normalize Data
scaler = StandardScaler()
df_normalized = scaler.fit_transform(df_sparse_mat)

# Step 3: Apply K-means Clustering
num_clusters = 16  # You can adjust this based on your dataset
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
user_clusters = kmeans.fit_predict(df_normalized)

# Add clusters to the sparse matrix DataFrame
df_sparse_mat['Cluster'] = user_clusters
print("Assigned Clusters:", df_sparse_mat['Cluster'].value_counts())

# Step 4: Recommend Movies Based on Cluster
def recommend_movies_kmeans(user_id, num_recommendations=10):
    # Get user's cluster
    user_cluster = df_sparse_mat.loc[user_id, 'Cluster']

    # Filter movies rated by users in the same cluster
    similar_users = df_sparse_mat[df_sparse_mat['Cluster'] == user_cluster].index
    cluster_movies = df[df['Cust_Id'].isin(similar_users)]

    # Calculate average rating for each movie within the cluster
    movie_recommendations = cluster_movies.groupby('Movie_Id')['Rating'].mean().sort_values(ascending=False)

    # Exclude movies the user has already rated
    user_rated_movies = df[df['Cust_Id'] == user_id]['Movie_Id'].unique()
    movie_recommendations = movie_recommendations[~movie_recommendations.index.isin(user_rated_movies)]

    # Join with movie titles for easy interpretation
    recommendations = movie_recommendations.head(num_recommendations).to_frame('Average_Rating').join(df_title)
    return recommendations[['Name', 'Average_Rating']]


recommended_movies = recommend_movies_kmeans(1932594)
print("Recommended Movies for User 1932594:")
print(recommended_movies)

Sparse Matrix Shape: (95325, 900)
Assigned Clusters: Cluster
8     15378
12    12047
2     10164
1      9672
10     9350
15     7237
6      4842
9      4355
4      4106
3      4080
7      3154
5      2945
11     2899
13     2315
14     1432
0      1349
Name: count, dtype: int64
Recommended Movies for User 1932594:
                                       Name  Average_Rating
Movie_Id                                                   
3456                         Lost: Season 1        4.607143
2162                          CSI: Season 1        4.511475
3864                          Batman Begins        4.463801
3046      The Simpsons: Treehouse of Horror        4.379965
1625            Aliens: Collector's Edition        4.342193
1256            The Best of Friends: Vol. 4        4.337912
2430             Alien: Collector's Edition        4.335840
4098                        Angel: Season 4        4.328571
1441                Stargate SG-1: Season 2        4.283582
1020                 The

In [16]:
# Function to calculate RMSE and MAE for a small number of users
from sklearn.metrics import mean_squared_error, mean_absolute_error
def evaluate_recommendations_small(df, df_sparse_mat, num_users=100, num_clusters=16):
    # Ensure the number of clusters is <= the number of users
    if num_clusters > num_users:
        num_clusters = num_users

    # Step 1: Select a subset of users
    selected_users = df['Cust_Id'].unique()[:num_users]
    df_small = df[df['Cust_Id'].isin(selected_users)]

    # Step 2: Split Data into Train/Test
    test_set = df_small.sample(frac=0.20, random_state=42)  # 20% test data
    train_set = df_small.drop(test_set.index)

    # Recreate the pivot table and normalize based on the training data
    df_sparse_mat_train = pd.pivot_table(train_set, values='Rating', index='Cust_Id', columns='Movie_Id').fillna(0)
    scaler = StandardScaler()
    df_normalized_train = scaler.fit_transform(df_sparse_mat_train)

    # Apply K-Means on the training data
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    user_clusters = kmeans.fit_predict(df_normalized_train)
    df_sparse_mat_train['Cluster'] = user_clusters

    # Step 3: Generate Predictions for Test Data
    predictions = []
    actuals = []

    for index, row in test_set.iterrows():
        user_id = row['Cust_Id']
        movie_id = row['Movie_Id']
        actual_rating = row['Rating']

        # Skip if user_id not in the training sparse matrix
        if user_id not in df_sparse_mat_train.index:
            continue

        # Get the user's cluster
        user_cluster = df_sparse_mat_train.loc[user_id, 'Cluster']

        # Calculate the average rating for the movie within the cluster
        similar_users = df_sparse_mat_train[df_sparse_mat_train['Cluster'] == user_cluster].index
        cluster_movies = train_set[train_set['Cust_Id'].isin(similar_users)]
        predicted_rating = cluster_movies[cluster_movies['Movie_Id'] == movie_id]['Rating'].mean()

        # Skip if no prediction can be made
        if np.isnan(predicted_rating):
            continue

        # Append to predictions and actuals
        predictions.append(predicted_rating)
        actuals.append(actual_rating)

    # Step 4: Calculate RMSE and MAE
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)

    return rmse, mae

# Call the function for 10 users with adjusted clusters
rmse, mae = evaluate_recommendations_small(df, df_sparse_mat, num_users=1000, num_clusters=5)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

RMSE: 0.9980642882072398
MAE: 0.7943681794445698


KNN

In [17]:
from sklearn.neighbors import NearestNeighbors

# Step 5: Apply K-nearest Neighbors (KNN) for Recommendation
def recommend_movies_knn(user_id, num_neighbors=5, num_recommendations=10):
    # Initialize the KNN model
    knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_model.fit(df_normalized)

    # Find K-nearest neighbors for the specified user
    user_index = df_sparse_mat.index.get_loc(user_id)  # Get the index of the user
    distances, indices = knn_model.kneighbors([df_normalized[user_index]], n_neighbors=num_neighbors + 1)  # +1 to skip the user itself

    # Get indices of similar users (excluding the user itself)
    similar_users_indices = indices.flatten()[1:]  # Skip the first index (the user itself)
    similar_users = df_sparse_mat.index[similar_users_indices]

    # Get movies rated by similar users
    similar_users_movies = df[df['Cust_Id'].isin(similar_users)]

    # Calculate average rating for each movie from similar users
    movie_recommendations = similar_users_movies.groupby('Movie_Id')['Rating'].mean().sort_values(ascending=False)

    # Exclude movies already rated by the user
    user_rated_movies = df[df['Cust_Id'] == user_id]['Movie_Id'].unique()
    movie_recommendations = movie_recommendations[~movie_recommendations.index.isin(user_rated_movies)]

    # Join with movie titles for easy interpretation
    recommendations = movie_recommendations.head(num_recommendations).to_frame('Average_Rating').join(df_title)
    return recommendations[['Name', 'Average_Rating']]

# Example: Recommend movies for user 785314 using KNN
recommended_movies_knn = recommend_movies_knn(1932594)
print("KNN Recommended Movies for User 1932594:")
print(recommended_movies_knn)

KNN Recommended Movies for User 1932594:
                                              Name  Average_Rating
Movie_Id                                                          
2551                                        Trigun             5.0
3879                            Ghost in the Shell             5.0
1026                                 Silver Bullet             5.0
3456                                Lost: Season 1             5.0
1256                   The Best of Friends: Vol. 4             5.0
3864                                 Batman Begins             4.8
1625                   Aliens: Collector's Edition             4.8
1853                                    Death Wish             4.8
2430                    Alien: Collector's Edition             4.8
1803      Alien: Resurrection: Collector's Edition             4.8


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Function to calculate RMSE and MAE using KNN
def evaluate_recommendations_knn_small(df, num_users=1000, num_neighbors=5):
    # Step 1: Select a subset of users
    selected_users = df['Cust_Id'].unique()[:num_users]
    df_small = df[df['Cust_Id'].isin(selected_users)]

    # Step 2: Split Data into Train/Test
    test_set = df_small.sample(frac=0.20, random_state=42)  # 20% test data
    train_set = df_small.drop(test_set.index)

    # Recreate the pivot table and normalize based on the training data
    df_sparse_mat_train = pd.pivot_table(train_set, values='Rating', index='Cust_Id', columns='Movie_Id').fillna(0)
    scaler = StandardScaler()
    df_normalized_train = scaler.fit_transform(df_sparse_mat_train)

    # Initialize the KNN model
    knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_model.fit(df_normalized_train)

    # Step 3: Generate Predictions for Test Data
    predictions = []
    actuals = []

    for index, row in test_set.iterrows():
        user_id = row['Cust_Id']
        movie_id = row['Movie_Id']
        actual_rating = row['Rating']

        # Skip if user_id not in the training sparse matrix
        if user_id not in df_sparse_mat_train.index:
            continue

        # Find K-nearest neighbors
        user_index = df_sparse_mat_train.index.get_loc(user_id)
        distances, indices = knn_model.kneighbors(
            [df_normalized_train[user_index]], n_neighbors=num_neighbors + 1
        )  # +1 to include the user itself

        # Get indices of similar users (excluding the user itself)
        similar_users_indices = indices.flatten()[1:]
        similar_users = df_sparse_mat_train.index[similar_users_indices]

        # Get ratings for the movie from similar users
        similar_users_movies = train_set[train_set['Cust_Id'].isin(similar_users)]
        predicted_rating = similar_users_movies[similar_users_movies['Movie_Id'] == movie_id]['Rating'].mean()

        # Skip if no prediction can be made
        if np.isnan(predicted_rating):
            continue

        # Append to predictions and actuals
        predictions.append(predicted_rating)
        actuals.append(actual_rating)

    # Step 4: Calculate RMSE and MAE
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)

    return rmse, mae

# Call the function for 10 users with KNN
rmse, mae = evaluate_recommendations_knn_small(df, num_users=1000, num_neighbors=5)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

RMSE: 1.1515460667912907
MAE: 0.8853002070393375
